In [2]:
import numpy as np
import os
import pandas as pd

In [15]:
S0 = 145.0              # spot stock price
K = 120.0               # strike
T = 4/52                 # maturity 
r = 0.0154                 # risk free rate 
sig = 0.6               # diffusion coefficient or volatility
N = 4                   # number of periods or number of time steps  
payoff = "call"          # payoff

In [16]:
dt = float(T) / N                             # Delta t
u = np.exp(sig * np.sqrt(dt))                 # up factor
d = 1.0 / u    

In [17]:
S = np.zeros((N + 1, N + 1))
S[0, 0] = S0
z = 1
for t in range(1, N + 1):
    for i in range(z):
        S[i, t] = S[i, t-1] * u
        S[i+1, t] = S[i, t-1] * d
    z += 1

In [18]:

S

array([[145.        , 157.58086764, 171.25330929, 186.11203493,
        202.25997202],
       [  0.        , 133.42355779, 145.        , 157.58086764,
        171.25330929],
       [  0.        ,   0.        , 122.77135015, 133.42355779,
        145.        ],
       [  0.        ,   0.        ,   0.        , 112.96958849,
        122.77135015],
       [  0.        ,   0.        ,   0.        ,   0.        ,
        103.9503753 ]])

In [19]:
a = np.exp(r * dt)    # risk free compound return
p = (a - d)/ (u - d)  # risk neutral up probability
q = 1.0 - p           # risk neutral down probability
p

0.48098860964322865

In [22]:
S_T = S[:,-1]
V = np.zeros((N + 1, N + 1))
if payoff =="call":
    V[:,-1] = np.maximum(S_T-K, 0.0)
elif payoff =="put":
    V[:,-1] = np.maximum(K-S_T, 0.0)
V

array([[ 0.        ,  0.        ,  0.        ,  0.        , 82.25997202],
       [ 0.        ,  0.        ,  0.        ,  0.        , 51.25330929],
       [ 0.        ,  0.        ,  0.        ,  0.        , 25.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  2.77135015],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

In [23]:
S_T = S[:,-1]
S_T

array([202.25997202, 171.25330929, 145.        , 122.77135015,
       103.9503753 ])

In [24]:
for j in range(N-1, -1, -1):
    for i in range(j+1):
        V[i,j] = np.exp(-r*dT) * (p * V[i,j + 1] + q * V[i + 1,j + 1])
V

array([[25.80119821, 37.14448208, 50.83022892, 65.8283731 , 82.25997202],
       [ 0.        , 15.54476885, 24.82967895, 37.43488293, 51.25330929],
       [ 0.        ,  0.        ,  7.09421374, 13.39414415, 25.        ],
       [ 0.        ,  0.        ,  0.        ,  1.32616272,  2.77135015],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

In [27]:

print('European ' + payoff, str( V[0,0]))


European call 26.05585713993444


In [26]:
if payoff =="call":
    for j in range(N-1, -1, -1):
        for i in range(j+1):
            V[i,j] = np.maximum(S[i,j] - K,np.exp(-r*dT) * (p * V[i,j + 1] + q * V[i + 1,j + 1]))
elif payoff =="put":
    for j in range(N-1, -1, -1):
        for i in range(j+1):
            V[i,j] = np.maximum(K - S[i,j],np.exp(-r*dT) * (p * V[i,j + 1] + q * V[i + 1,j + 1]))
V

array([[26.05585714, 37.58086764, 51.25330929, 66.11203493, 82.25997202],
       [ 0.        , 15.63353965, 25.        , 37.58086764, 51.25330929],
       [ 0.        ,  0.        ,  7.10828892, 13.42355779, 25.        ],
       [ 0.        ,  0.        ,  0.        ,  1.32616272,  2.77135015],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

In [28]:
print('American ' + payoff, str( V[0,0]))

American call 26.05585713993444


In [29]:
def mcs_simulation_np(p):
    M = p
    I = p
    dt = T / M 
    S = np.zeros((M + 1, I))
    S[0] = S0 
    rn = np.random.standard_normal(S.shape) 
    for t in range(1, M + 1): 
        S[t] = S[t-1] * np.exp((r - sigma ** 2 / 2) * dt + sigma * np.sqrt(dt) * rn[t]) 
    return S

In [30]:
T = 1
r = 0.1
sigma = 0.2
S0 = 100
K = 100

In [31]:
S = mcs_simulation_np(1000)

In [33]:
S

array([[100.        , 100.        , 100.        , ..., 100.        ,
        100.        , 100.        ],
       [ 99.55778076,  99.84503771,  98.90536215, ...,  99.92740816,
         98.56713208, 100.78087218],
       [ 99.29711316, 100.22301431,  99.30391847, ...,  99.78780496,
         98.7062489 , 100.73879221],
       ...,
       [120.73560601, 111.82170366, 129.36144718, ...,  87.87081205,
         94.60048259,  74.12760455],
       [119.85861407, 112.05762485, 129.16907625, ...,  88.01266168,
         94.01359759,  74.772322  ],
       [119.79606639, 112.99940554, 129.27222773, ...,  87.14294743,
         92.96020255,  74.60319438]])

In [34]:
S = np.transpose(S)
S

array([[100.        ,  99.55778076,  99.29711316, ..., 120.73560601,
        119.85861407, 119.79606639],
       [100.        ,  99.84503771, 100.22301431, ..., 111.82170366,
        112.05762485, 112.99940554],
       [100.        ,  98.90536215,  99.30391847, ..., 129.36144718,
        129.16907625, 129.27222773],
       ...,
       [100.        ,  99.92740816,  99.78780496, ...,  87.87081205,
         88.01266168,  87.14294743],
       [100.        ,  98.56713208,  98.7062489 , ...,  94.60048259,
         94.01359759,  92.96020255],
       [100.        , 100.78087218, 100.73879221, ...,  74.12760455,
         74.772322  ,  74.60319438]])

In [35]:
p = np.mean(np.maximum(K - S[:,-1],0))
print('European put', str(p))

European put 4.487269433346493


In [36]:
c = np.mean(np.maximum(S[:,-1] - K,0))
print('European call', str(c))

European call 14.67858784572602
